In [2]:
!pip install -U spacy




     --------------------------------------- 12.1/12.1 MB 11.7 MB/s eta 0:00:00
     ------------------------------------- 381.9/381.9 kB 12.0 MB/s eta 0:00:00
     ---------------------------------------- 181.6/181.6 kB ? eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 18.9 MB/s eta 0:00:00
     ---------------------------------------- 122.2/122.2 kB ? eta 0:00:00
     ---------------------------------------- 45.9/45.9 kB ? eta 0:00:00
     ------------------------------------- 481.9/481.9 kB 31.4 MB/s eta 0:00:00
     ---------------------------------------- 50.1/50.1 kB ? eta 0:00:00
     ---------------------------------------- 1.9/1.9 MB 15.0 MB/s eta 0:00:00
     ---------------------------------------- 6.6/6.6 MB 13.2 MB/s eta 0:00:00
     ---------------------------------------- 45.0/45.0 kB ? eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Success

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.9.0 which is incompatible.
Traceback (most recent call last):
  File "C:\Users\ZEAL INSTITUTE\anaconda3\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\ZEAL INSTITUTE\anaconda3\lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "C:\Users\ZEAL INSTITUTE\anaconda3\lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

During handling of the above

In [7]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 12.8/12.8 MB 2.0 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
data = 'Aniket,Phadnis,aniket.xxx@gmail.com,XYX Ltd,8888888888'

In [3]:
import spacy

In [4]:
nlp = spacy.load("en_core_web_sm")


In [5]:
doc = nlp(data)

In [6]:
for entity in doc.ents:
    print(entity.text, entity.label_)

Phadnis PERSON
Virtusa PERSON


In [7]:
doc

Aniket,Phadnis,aniket.phadnis@gmail.com,Virtusa Ltd,8971257351

#Update the NER

In [155]:
nlp = spacy.load('en_core_web_sm')

In [156]:
ner = nlp.get_pipe('ner')

In [157]:
import pandas as pd

In [158]:
raw_data = pd.read_csv('data/training_data.csv')

In [159]:
raw_data_col = raw_data.columns

In [160]:
Train_data = []

In [161]:
raw_data_formatted = [tuple(row) for row in raw_data.itertuples(index=False)]

In [162]:
raw_data_text = []
for data in raw_data_formatted:
    csv_string = '|'.join(data)
    raw_data_text.append(csv_string)

In [163]:
for i in range(len(raw_data_text)):
    list_row = []
    list_row.append(raw_data_text[i])
    entities = {}
    entities['entities'] = []
    values = raw_data_text[i].split('|')
    start_index = 0
    for index,value in enumerate(values):
        end_index = start_index + len(value)
        entities['entities'].append((start_index,end_index,raw_data_col[index]))
        start_index = end_index +1
    
    list_row.append(entities)
    Train_data.append(tuple(list_row))

In [164]:
Train_data[0]

('Keefe Cook|1-533-202-6374|ridiculus@icloud.org|Ap #332-2746 Cras St.|J8H 4G3|Chile|WILLIAM GRAY & COMPANY LIMITED',
 {'entities': [(0, 10, 'name'),
   (11, 25, 'phone'),
   (26, 46, 'email'),
   (47, 68, 'address'),
   (69, 76, 'postalZip'),
   (77, 82, 'GPE'),
   (83, 113, 'ORG')]})

In [165]:
#Adding the labels to NER

for _,annotation in Train_data:
    for ent in annotation.get('entities'):
        ner.add_label(ent[2])

In [166]:
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [168]:
import re
from spacy.tokenizer import Tokenizer
def custom_tokenizer(nlp):
    infix_re = re.compile(r'''[|]''')  # Only the pipe character

    return Tokenizer(nlp.vocab, prefix_search=None,
                                suffix_search=None,
                                infix_finditer=infix_re.finditer,
                                token_match=None)

# Set the tokenizer to the custom tokenizer
nlp.tokenizer = custom_tokenizer(nlp)

doc = nlp("Troy Allen|(386) 785-1765|vel.arcu@aol.com|Ap #504-7516 Eu Street|8006|United Kingdom|WEST CHESHIRE & NORTH WALES CHAMBER OF COMMERCE")
for token in doc:
    print(token.text)

Troy
Allen
|
(386)
785-1765
|
vel.arcu@aol.com
|
Ap
#504-7516
Eu
Street
|
8006
|
United
Kingdom
|
WEST
CHESHIRE
&
NORTH
WALES
CHAMBER
OF
COMMERCE


In [169]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path
#from spacy.training import offsets_to_biluo_tags


# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):
    for iteration in range(30):
        random.shuffle(Train_data)
        losses = {}

        # Create batches of Example objects
        batches = minibatch(Train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            examples = [Example.from_dict(nlp.make_doc(text), annotations) for text, annotations in batch]

            # Update the model with the batch of Example objects
            nlp.update(examples, drop=0.5, losses=losses)

            print("Losses", losses)

Losses {'ner': 51.891173044972724}
Losses {'ner': 97.33399388000367}
Losses {'ner': 141.811567231036}
Losses {'ner': 184.61173279538707}
Losses {'ner': 228.37229422534074}
Losses {'ner': 270.97301548354517}
Losses {'ner': 311.8976072372061}
Losses {'ner': 361.00053882981956}
Losses {'ner': 402.28257019490775}
Losses {'ner': 441.7456612286587}
Losses {'ner': 481.8829737199967}
Losses {'ner': 522.0379658550928}
Losses {'ner': 568.0411470149966}
Losses {'ner': 606.3068838900044}
Losses {'ner': 644.1010676089763}
Losses {'ner': 688.8348616427365}
Losses {'ner': 731.4900684139075}
Losses {'ner': 768.9963017558994}
Losses {'ner': 810.2980744920287}
Losses {'ner': 848.079924409868}
Losses {'ner': 891.1529086871387}
Losses {'ner': 924.4360585134791}
Losses {'ner': 962.3060188626487}
Losses {'ner': 999.0133348691825}
Losses {'ner': 1033.97141111154}
Losses {'ner': 1067.479770097657}
Losses {'ner': 1098.7584274109477}
Losses {'ner': 1134.316886221633}
Losses {'ner': 1167.6871363988535}
Losses {'

Losses {'ner': 1069.7200859373709}
Losses {'ner': 1071.9426403104003}
Losses {'ner': 1077.6812903101152}
Losses {'ner': 1078.9884094435033}
Losses {'ner': 1080.2951445006695}
Losses {'ner': 1082.0981226470592}
Losses {'ner': 1084.4157230338694}
Losses {'ner': 1084.694649265288}
Losses {'ner': 1087.8741522271066}
Losses {'ner': 1.672301424299202}
Losses {'ner': 4.374261041855904}
Losses {'ner': 11.16678805040882}
Losses {'ner': 13.766813941468039}
Losses {'ner': 14.717428469541382}
Losses {'ner': 20.295275096187304}
Losses {'ner': 23.115594608374927}
Losses {'ner': 26.7950857926653}
Losses {'ner': 29.054914469561748}
Losses {'ner': 29.900695598943894}
Losses {'ner': 35.965893519943776}
Losses {'ner': 43.38360368001628}
Losses {'ner': 46.65584583649032}
Losses {'ner': 48.414280506894634}
Losses {'ner': 51.94224189540513}
Losses {'ner': 55.16537826177714}
Losses {'ner': 58.41941298309791}
Losses {'ner': 59.037137024059426}
Losses {'ner': 59.80890133664117}
Losses {'ner': 62.40311779790795

Losses {'ner': 75.19212140709416}
Losses {'ner': 75.1921462995637}
Losses {'ner': 75.21404101469237}
Losses {'ner': 75.22818950883146}
Losses {'ner': 76.77099450258766}
Losses {'ner': 77.07188096590964}
Losses {'ner': 77.07195882410842}
Losses {'ner': 77.07216837527339}
Losses {'ner': 77.07243151418425}
Losses {'ner': 77.55261201085014}
Losses {'ner': 77.73289890394253}
Losses {'ner': 77.73305767757071}
Losses {'ner': 77.83089057137425}
Losses {'ner': 77.83333604026632}
Losses {'ner': 80.00170823654771}
Losses {'ner': 80.00182971454797}
Losses {'ner': 80.00183048417381}
Losses {'ner': 82.00237019116987}
Losses {'ner': 83.68097828418337}
Losses {'ner': 8.027814541863603e-05}
Losses {'ner': 0.004213718704900018}
Losses {'ner': 1.8606585635966244}
Losses {'ner': 1.8689385285828952}
Losses {'ner': 2.466589652082482}
Losses {'ner': 2.4665994780436926}
Losses {'ner': 3.4525267806064965}
Losses {'ner': 3.8064228999529957}
Losses {'ner': 3.809954153900943}
Losses {'ner': 3.820721105693207}
Los

Losses {'ner': 34.356115598502114}
Losses {'ner': 34.35611732640619}
Losses {'ner': 34.423883198291634}
Losses {'ner': 34.42953487716388}
Losses {'ner': 34.42953757266725}
Losses {'ner': 34.430347945292304}
Losses {'ner': 34.43034959061663}
Losses {'ner': 34.43062592339885}
Losses {'ner': 34.430638825406945}
Losses {'ner': 34.57503698431163}
Losses {'ner': 34.57522373879582}
Losses {'ner': 34.57537785241401}
Losses {'ner': 35.04746742395882}
Losses {'ner': 35.04751219995636}
Losses {'ner': 35.05566246102367}
Losses {'ner': 35.055941817719145}
Losses {'ner': 35.419209199853405}
Losses {'ner': 35.419234060032764}
Losses {'ner': 36.596994495473034}
Losses {'ner': 36.59703105438703}
Losses {'ner': 36.597135064161385}
Losses {'ner': 36.59716242309877}
Losses {'ner': 36.62273902690533}
Losses {'ner': 36.62273923982472}
Losses {'ner': 36.622740036536214}
Losses {'ner': 36.624593704262}
Losses {'ner': 36.62459567657559}
Losses {'ner': 36.62460082329978}
Losses {'ner': 0.0007733293547443911}
Lo

Losses {'ner': 14.934879133799086}
Losses {'ner': 14.934879189665677}
Losses {'ner': 14.934888563105206}
Losses {'ner': 14.93488862539512}
Losses {'ner': 14.934918193251312}
Losses {'ner': 14.934918977100802}
Losses {'ner': 14.93491958854788}
Losses {'ner': 14.935315816755137}
Losses {'ner': 14.93531581845764}
Losses {'ner': 16.49094246610418}
Losses {'ner': 16.490942601230877}
Losses {'ner': 16.773617152955428}
Losses {'ner': 16.773677033140007}
Losses {'ner': 17.514299231565293}
Losses {'ner': 17.514299688698127}
Losses {'ner': 17.514310896245018}
Losses {'ner': 17.51432599211821}
Losses {'ner': 17.514329347386745}
Losses {'ner': 17.514505559467377}
Losses {'ner': 17.514506793315817}
Losses {'ner': 17.514506802363133}
Losses {'ner': 18.83639125941999}
Losses {'ner': 18.836632173143055}
Losses {'ner': 18.836632179692817}
Losses {'ner': 18.836632191497404}
Losses {'ner': 18.868587869870975}
Losses {'ner': 18.868587870040475}
Losses {'ner': 18.86858852499256}
Losses {'ner': 18.868588862

...

Losses {'ner': 0.0016859477535617362}
Losses {'ner': 0.0016859516314924896}
Losses {'ner': 0.0016859516328565894}
Losses {'ner': 0.001685951643615516}
Losses {'ner': 0.0016859518816388045}
Losses {'ner': 0.0016859518884772327}
Losses {'ner': 0.0016861171863005995}
Losses {'ner': 0.00168611724714979}
Losses {'ner': 0.0016861172472163886}
Losses {'ner': 0.0016871244925833655}
Losses {'ner': 0.0016871245017652282}
Losses {'ner': 0.001706971335221924}
Losses {'ner': 0.0020293433475392845}
Losses {'ner': 0.0020293434427520935}
Losses {'ner': 0.002029343443382595}
Losses {'ner': 0.002029354305117234}
Losses {'ner': 0.0011526944588718304}
Losses {'ner': 0.0011527035983683865}
Losses {'ner': 0.0011527212374748115}
Losses {'ner': 0.0011527212727174547}
Losses {'ner': 0.0011527212742995058}
Losses {'ner': 0.0011527213035468194}
Losses {'ner': 0.0011527214183870375}
Losses {'ner': 0.0011527604543357462}
Losses {'ner': 0.0011527605223666797}
Losses {'ner': 0.0011527611242299694}
Losses {'ner': 0.0

Losses {'ner': 2.5296182331916017}
Losses {'ner': 2.529618233659879}
Losses {'ner': 2.5296182338316413}
Losses {'ner': 2.5296197692041}
Losses {'ner': 2.529619778776892}
Losses {'ner': 2.536270023992609}
Losses {'ner': 2.536270024010031}
Losses {'ner': 2.536270024013443}
Losses {'ner': 2.5362700266070246}
Losses {'ner': 2.536270026607429}
Losses {'ner': 2.536270026610825}
Losses {'ner': 2.536270026610834}
Losses {'ner': 2.5362700826450952}
Losses {'ner': 2.536270121582153}
Losses {'ner': 2.6094704889738645}
Losses {'ner': 2.6094704889762284}
Losses {'ner': 2.6094704890616622}
Losses {'ner': 2.6094704890617515}
Losses {'ner': 2.609470490062596}
Losses {'ner': 2.6095233173055568}
Losses {'ner': 2.829883581491059}
Losses {'ner': 2.832181703033609}
Losses {'ner': 2.832181703063713}
Losses {'ner': 2.8321817030706806}
Losses {'ner': 2.8321817031269756}
Losses {'ner': 2.8321817031274534}
Losses {'ner': 2.8321817034796126}
Losses {'ner': 2.8321817034801025}
Losses {'ner': 2.8321817062832126}
L

Losses {'ner': 3.7857810381667756}
Losses {'ner': 3.7857810381669426}
Losses {'ner': 3.7857810572960537}
Losses {'ner': 3.785781057296792}
Losses {'ner': 3.785781057299185}
Losses {'ner': 3.7857810575303743}
Losses {'ner': 3.785781125546046}
Losses {'ner': 3.78578112562516}
Losses {'ner': 3.7857811265619046}
Losses {'ner': 3.7857811265619286}
Losses {'ner': 3.7857938758234866}
Losses {'ner': 3.7857938758345835}
Losses {'ner': 3.834849310328087}
Losses {'ner': 3.8348493103282006}
Losses {'ner': 3.8348493103282055}
Losses {'ner': 3.8348493170138997}
Losses {'ner': 3.834849317030713}
Losses {'ner': 3.8348493170333287}
Losses {'ner': 3.8348493170421643}
Losses {'ner': 3.834849317042166}
Losses {'ner': 3.8348493171936164}
Losses {'ner': 3.834849317204263}
Losses {'ner': 3.834849317204405}
Losses {'ner': 3.834849317205585}
Losses {'ner': 3.8348494836610203}
Losses {'ner': 3.8348494836610714}
Losses {'ner': 3.83484948366119}
Losses {'ner': 3.8348494891605145}
Losses {'ner': 3.8348499487137415

Losses {'ner': 0.42810307776609086}
Losses {'ner': 0.42810308021548377}
Losses {'ner': 0.4281030802278766}
Losses {'ner': 0.4281030802331738}
Losses {'ner': 0.42810308024731186}
Losses {'ner': 0.4281030802474656}
Losses {'ner': 0.4281030813489742}
Losses {'ner': 0.4281030813490031}
Losses {'ner': 0.4281030815784599}
Losses {'ner': 0.4281030815799071}
Losses {'ner': 0.428103081787226}
Losses {'ner': 0.4281030820761099}
Losses {'ner': 0.42810308207631115}
Losses {'ner': 0.42810316970553286}
Losses {'ner': 0.4281031697072852}
Losses {'ner': 0.42810354474749707}
Losses {'ner': 0.42810354481297347}
Losses {'ner': 0.4292529178360752}
Losses {'ner': 0.42925292368778706}
Losses {'ner': 0.4292531149283545}
Losses {'ner': 0.42925311493241153}
Losses {'ner': 0.4292531161410787}
Losses {'ner': 0.4292542939722127}
Losses {'ner': 0.42925429397241777}
Losses {'ner': 5.684389931006363}
Losses {'ner': 8.279027798310233}
Losses {'ner': 8.279027798310329}
Losses {'ner': 8.279027798310807}
Losses {'ner': 

Losses {'ner': 1.2467339954871148}
Losses {'ner': 1.2467339954896206}
Losses {'ner': 1.2467339955461452}
Losses {'ner': 1.2467339985063697}
Losses {'ner': 1.2467339995576545}
Losses {'ner': 1.2467497912760042}
Losses {'ner': 1.2467497912845718}
Losses {'ner': 1.2467497913767436}
Losses {'ner': 1.246749791399395}
Losses {'ner': 1.246750281983808}
Losses {'ner': 1.2480421639991415}
Losses {'ner': 1.2480421640107087}
Losses {'ner': 1.2480421640108432}
Losses {'ner': 1.2480421640716874}
Losses {'ner': 1.254216105875982}
Losses {'ner': 1.2542161060422947}
Losses {'ner': 1.254216106042732}
Losses {'ner': 1.2542161061105768}
Losses {'ner': 1.2542161061410233}
Losses {'ner': 1.254216108592041}
Losses {'ner': 1.2542161215130558}
Losses {'ner': 1.2542161215309968}
Losses {'ner': 1.2542161448954923}
Losses {'ner': 1.2542161449017009}
Losses {'ner': 1.254325485128781}
Losses {'ner': 1.2543273579858247}
Losses {'ner': 1.2543273581243892}
Losses {'ner': 1.2543273581293912}
Losses {'ner': 1.254327358

In [176]:
text = 'YORKSHIRE POWER LTD|eget.odio@google.net|6 In Avenue|H7 7TR|Vietnam|Angela Drake|(717) 789-3399'

doc = nlp(text)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('YORKSHIRE POWER LTD', 'ORG'), ('eget.odio@google.net', 'email'), ('6 In Avenue', 'address'), ('H7 7TR', 'postalZip'), ('Vietnam', 'GPE'), ('Angela Drake', 'name'), ('(717) 789-3399', 'phone')]
